라이브러리 불러오기 / 파일 읽어와 형변환 

In [1]:
#라이브러리 불러오기
import pandas as pd
import numpy as np

#파일 불러오기 및 형 변환
selloutData = pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")
selloutData = selloutData.astype({"REGIONID":str,
                                  "PRODUCT":str,
                                  "YEARWEEK":str,
                                  "QTY":float})

In [2]:
#if QTY_NEW < 0 : 0 , else: Qty값 
selloutData["QTY_NEW"] = np.where( selloutData.QTY < 0,0,selloutData["QTY"] )
#YEARWEEK.str로 문자열 인덱스로 활용 YEAR / WEEK 정의 
selloutData["YEAR"] , selloutData["WEEK"] = selloutData.YEARWEEK.str[:-2] , selloutData.YEARWEEK.str[-2:]
#정의한 WEEK가 52이하인 값을 조회해 변수에 대입  /selloutData.WEEK.max()값을 얻고 isin을 활용해도 좋을 듯 하다.
refinedSelloutData = selloutData.loc[ selloutData.WEEK.astype(int) <=52 ]

In [3]:
refinedSelloutData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,02
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,02
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,02
3,A60,PRODUCT47,201402,0.0,0.0,2014,02
4,A60,PRODUCT56,201402,23.0,23.0,2014,02
...,...,...,...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0,824.0,2016,30
124654,A10,PRODUCT56,201630,275.0,275.0,2016,30
124655,A10,PRODUCT61,201630,0.0,0.0,2016,30
124656,A10,PRODUCT12,201630,15021.0,15021.0,2016,30


In [4]:
#group by code
groupKey = ["REGIONID","PRODUCT","YEAR"] 
#groupby groupkey 그룹별 qty_new값의 평균을 나타냄 / restart index , inplace
groupData = refinedSelloutData.groupby(groupKey)["QTY_NEW"].agg(["mean"]).reset_index()
#mean컬럼 -> qty_new로 재설정 
groupData = groupData.rename(columns = {"mean":"QTY_MEAN"})
groupData

,REGIONID,PRODUCT,YEAR,QTY_MEAN
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,5.711538
...,...,...,...,...
2377,A77,PRODUCT1,2015,3030.019231
2378,A77,PRODUCT1,2016,3375.326923
2379,A77,PRODUCT12,2014,2035.788462
2380,A77,PRODUCT12,2015,3540.980769


In [5]:
#데이터 조인
#조인키를 미리 설정하여 조인
joinKey = ("REGIONID","PRODUCT","YEAR")
mergedData = pd.merge(left=refinedSelloutData,
        right=groupData,
        left_on=joinKey,
        right_on=joinKey,
        how="outer")

mergedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,QTY_MEAN
0,A60,PRODUCT4,201402,71.0,71.0,2014,02,29.134615
1,A60,PRODUCT4,201401,16.0,16.0,2014,01,29.134615
2,A60,PRODUCT4,201403,51.0,51.0,2014,03,29.134615
3,A60,PRODUCT4,201406,19.0,19.0,2014,06,29.134615
4,A60,PRODUCT4,201404,21.0,21.0,2014,04,29.134615
...,...,...,...,...,...,...,...,...
123859,A28,PRODUCT40,201612,0.0,0.0,2016,12,1.903846
123860,A28,PRODUCT40,201632,0.0,0.0,2016,32,1.903846
123861,A28,PRODUCT40,201631,1.0,1.0,2016,31,1.903846
123862,A28,PRODUCT40,201628,4.0,4.0,2016,28,1.903846


In [6]:
#데이터 정렬하기 sortKey/sort code
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]
mergedData = mergedData.sort_values(by=sortKey) 
mergedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,QTY_MEAN
10661,A00,PRODUCT34,201401,661.0,661.0,2014,01,275.961538
10662,A00,PRODUCT34,201402,679.0,679.0,2014,02,275.961538
10663,A00,PRODUCT34,201403,578.0,578.0,2014,03,275.961538
10660,A00,PRODUCT34,201404,532.0,532.0,2014,04,275.961538
10665,A00,PRODUCT34,201405,516.0,516.0,2014,05,275.961538
...,...,...,...,...,...,...,...,...
96903,A77,PRODUCT12,201648,4152.0,4152.0,2016,48,4837.153846
96893,A77,PRODUCT12,201649,5086.0,5086.0,2016,49,4837.153846
96904,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,4837.153846
96898,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,4837.153846


In [7]:
# SEASONALITY 컬럼에 계절성 지수 구한 뒤 넣어주기
mergedData["SEASONALITY"] = mergedData.QTY_NEW / mergedData.QTY_MEAN
mergedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,QTY_MEAN,SEASONALITY
10661,A00,PRODUCT34,201401,661.0,661.0,2014,01,275.961538,2.395261
10662,A00,PRODUCT34,201402,679.0,679.0,2014,02,275.961538,2.460488
10663,A00,PRODUCT34,201403,578.0,578.0,2014,03,275.961538,2.094495
10660,A00,PRODUCT34,201404,532.0,532.0,2014,04,275.961538,1.927805
10665,A00,PRODUCT34,201405,516.0,516.0,2014,05,275.961538,1.869826
...,...,...,...,...,...,...,...,...,...
96903,A77,PRODUCT12,201648,4152.0,4152.0,2016,48,4837.153846,0.858356
96893,A77,PRODUCT12,201649,5086.0,5086.0,2016,49,4837.153846,1.051445
96904,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,4837.153846,1.208562
96898,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,4837.153846,1.019815


In [10]:
#최종 [지역, 상품, 주차 ] 별 계절성 지수 평균값 산출 후 finalResult 변수에 담기 
groupKey = ["REGIONID","PRODUCT","WEEK"] 
finalResult  = mergedData.groupby(groupKey)["SEASONALITY"].agg(["mean"]).reset_index()
finalResult

,REGIONID,PRODUCT,WEEK,mean
0,A00,PRODUCT34,01,1.570782
1,A00,PRODUCT34,02,1.755540
2,A00,PRODUCT34,03,1.319460
3,A00,PRODUCT34,04,1.490298
4,A00,PRODUCT34,05,1.061909
5,A00,PRODUCT34,06,0.992711
